# H2 Weathermap Creation

The creation of weathermaps for H2 dataset is identical in nearly every way. The only differences relate to the size of the H2 data. For example, when reshaping the data, it will be reshaped to (180, 90) and not (360, 180) etc.
See `h1_wsmaps.ipynb` for all the details in the creation of these maps.

In [2]:
import os
import gc
import rfo
import time
import netCDF4
import numpy as np
import matplotlib as mpl
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.ticker import FormatStrFormatter, FixedFormatter
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
# Find path where data is located, initialize empty array to store data, 
# and determine how many years of data to plot
path = '/run/media/dtropf/Seagate Backup Plus Drive/H2_data/H2_'
data = np.zeros((12,12,180*90))
years = [str(i) for i in range(1984, 2013)]

# Open a year of data at a time, add to the running total, and free memory
# to load next year of data
for year in years:
    h1data = np.load(path+year+'.npy')
    data[:,:,:] = data[:,:,:]+h1data[:,:,:]
    del h1data
    gc.collect()

In [4]:
# Total gridbox count. Data summed along temporal and WS axis'.
total_data = np.sum(np.sum(data,axis=0),axis=0)

# Masked version of total_data used to find and ignore gridboxes without counts
masked_total = np.ma.masked_where(total_data<=0, total_data)

# Masked data summed along temporal axis. This ignores gridboxes without counts
# and obtains counts for gridboxes of each WS
masked_data = np.ma.masked_where(np.sum(data,axis=0)<=0, np.sum(data,axis=0))

# Intialize new array which stores the masked_data in RFO
# form by dividing each WS of masked_data by total_data
normed_data = np.ma.zeros((12,180*90))
for i in range(12):
    normed_data[i,:] = masked_data[i,:]/total_data

In [6]:
# Create custom colorbar
cf = open('cmap3.txt')
clist = [""+cnum+"" for cnum in cf.read().splitlines()]
cm = mpl.colors.LinearSegmentedColormap.from_list('custom', clist, N=256)
vmin = np.amin(normed_data)
vmax = np.amax(normed_data)*.7
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

In [7]:
# Compute RFO for each WS
rfo_list = [rfo.weighted_rfo(data) for data in normed_data]

# Use Cartopy and plot H2 data
projection = ccrs.PlateCarree(central_longitude=180)
axes_class = (GeoAxes, dict(map_projection=projection))

fig = plt.figure(figsize=(10,5.5))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(4, 3),
                axes_pad=0.6,
                cbar_pad=0.0,
                cbar_size='3%',
                cbar_mode = 'single',
                cbar_location = 'bottom',
                label_mode='', aspect='equal')  # note the empty label_mode

lats = np.arange(-90, 90)
lons = np.arange(-180, 180)

for i, ax in enumerate(axgr):
    ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
    ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.coastlines()
    ax.set_global()
    ax.set_title('WS '+str(i+1)+' RFO '+'{:.1f}%'.format(rfo_list[i]*100))
    extent=[-180,180,-90,90]
    im = ax.imshow((normed_data[i,:].reshape(180,90).T), cmap=cm, norm=norm, transform=projection, interpolation='none', extent=extent)

axgr.cbar_axes[0].colorbar(im)
xticks = [0,.2,.4,.6,vmax]
axgr.cbar_axes[0].set_xticks(xticks)
xticks = ['{:.2f}'.format(tick) for tick in xticks]
axgr.cbar_axes[0].xaxis.set_major_formatter(FixedFormatter(xticks))
fig.suptitle("H2 WS Maps", fontweight='extra bold', fontsize='x-large')
fig.tight_layout()
plt.show()

/home/dtropf/anaconda3/envs/cartopyTest/lib/python3.6/site-packages/matplotlib/figure.py:2022: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "
